In [ ]:
import asyncio
import zendriver as zd
import pandas as pd
from io import StringIO

async def main():
    browser = await zd.start(headless=True)
    print("Browser started.")
    
    # 1. Get the page
    #page = await browser.get("https://fbref.com/en/squads/18bb7c10/Arsenal-Stats")
    page = await browser.get("https://www.premierleague.com/en/clubs/3/arsenal/stats")
    
    # 2. Ensure the table is present (using select acts as a wait)
    # We don't need the element object itself, just to know it's there
    await page.select("#stats_standard_9")
    
    # 3. Get the HTML string by executing JavaScript
    # This runs inside the browser and returns the HTML string to Python
    table_html = await page.evaluate("document.querySelector('#stats_standard_9').outerHTML")
    
    if not table_html:
        print("Error: Could not retrieve table HTML.")
        await browser.stop()
        return

    # 4. Use Pandas to parse the HTML string
    # Wrap in StringIO to avoid pandas warnings
    dfs = pd.read_html(StringIO(table_html))
    df = dfs[0]

    # 5. Clean up the Multi-level Headers (Flattening)
    # This turns headers like ('Performance', 'Gls') into 'Performance_Gls'
    new_columns = []
    for col in df.columns.values:
        # col is a tuple like ('Performance', 'Gls')
        top_level = str(col[0]).strip()
        bottom_level = str(col[1]).strip()
        
        if "Unnamed" in top_level or not top_level:
            new_columns.append(bottom_level)
        else:
            new_columns.append(f"{top_level}_{bottom_level}")
    
    df.columns = new_columns

    # 6. Save to CSV
    csv_filename = "arsenal_stats.csv"
    df.to_csv(csv_filename, index=False)
    
    print(f"Successfully saved {len(df)} rows to {csv_filename}")
    print(df.head()) # Print first few rows to verify

    await browser.stop()

# Since you are in a notebook (Jupyter/IPython), use await main() directly
await main()

Browser started.
